<a href="https://colab.research.google.com/github/Vaibhavs10/translate-with-whisper/blob/main/whisper_en_to_any_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚠️ EXPERIMENTAL: Transcribe audio to any language w/ 🤗 Transformers

by: [Vaibhav (VB) Srivastav](https://twitter.com/reach_vb)

Heads-up, this is pretty much a hack, the model wasn't trained on this specific objective, so the results may not be as reliable.

🪄 The magic sauce is to force the model to transcribe in a particular language!

The environment setup is pretty much straightforward, we'll use `transformers` to load the [Whisper-large-v2](https://huggingface.co/openai/whisper-large-v2) checkpoint (`fp-16`) in a colab's free GPU! 🏃‍♂️

In [1]:
!pip -q install transformers datasets huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.7 MB/s eta 0:00:00


Note: We only need to authenticate for the purpose of accessing the [Common Voice dataset](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). You can safely ignore this if you are using your own audio dataset.

You can find your access token at: [hf.co/settings/token](httpS://hf.co/settings/token)

In [2]:
from huggingface_hub import notebook_login

notebook_login()

For the purpose of colab demo, we'll use a Whisper-large-v2 checkpoint in half-precision (`fp16`). If you have access to a larger GPU VRAM then remove the `torch_dtype` arg 🤗

In [3]:
import torch
from transformers import pipeline

whisper_asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
    torch_dtype=torch.float16,
    device="cuda:0"
    )

To keep things simple, we'll use the Common Voice dataset from the 🤗 Hub via `streaming` mode & resample the audio to 16KHz as expected by Whisper.

In [4]:
from datasets import load_dataset
from datasets import Audio

common_voice_en = load_dataset("mozilla-foundation/common_voice_11_0", "en",
                               revision="streaming",
                               split="test",
                               streaming=True,
                               use_auth_token=True)

common_voice_en = common_voice_en.cast_column("audio",
                                              Audio(sampling_rate=16000))

Alrighty! Let's listen to our audio file and take a look at its transcription!

In [5]:
import IPython.display as ipd

ipd.Audio(next(iter(common_voice_en))["audio"]["array"], rate=next(iter(common_voice_en))["audio"]["sampling_rate"]) # load a NumPy array

Reading metadata...: 16354it [00:00, 35998.45it/s]
Reading metadata...: 16354it [00:00, 63497.00it/s]


In [6]:
next(iter(common_voice_en))["sentence"]

Reading metadata...: 16354it [00:00, 30323.29it/s]


'Joe Keaton disapproved of films, and Buster also had reservations about the medium.'

Let's create a wee list of languages to transcribe too. 💥

In [7]:
list_of_languages = ["de", "it", "es", "nl", "fr"]

🪄 Time for the magic sauce, here, we essentially force Whisper to decode in the specific language. My hypothesis is that because Whisper was trained on 600K+ hours of data it is able to generalise fairly well.

👀 So the only change you'd need to make to make this happen would be to set the task as `transcribe` and change the target language.

In [8]:
for lang in list_of_languages:
    whisper_asr.model.config.forced_decoder_ids = (
        whisper_asr.tokenizer.get_decoder_prompt_ids(
            language=lang,
            task="transcribe"
            )
        )
    print(whisper_asr(next(iter(common_voice_en))["audio"]["array"])["text"])

Reading metadata...: 16354it [00:00, 46860.47it/s]
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 Joe Keaton hat Filme verabschiedet und Buster hatte auch Reservations über die Medien.


Reading metadata...: 16354it [00:00, 68016.46it/s]


 Joe Keaton ha disapprovato i film e Buster ha anche delle riservazioni sui media.


Reading metadata...: 16354it [00:00, 70042.55it/s]


 Joe Keaton disaproveció de los filmes y Buster también tenía reservaciones sobre la media.


Reading metadata...: 16354it [00:00, 69285.74it/s]


 Joe Keaton onverstaanbaar van de films en Buster had ook bewaarschuwingen over de media.


Reading metadata...: 16354it [00:00, 66251.33it/s]


 Joe Keaton a dénoncé les films et Buster avait également des réservations sur le médium.


Voila! Works quite okay, no? 🤗

Some of these translations are a bit out of the line, however, we can fix these with some neat generation techniques like [contrastive search](https://huggingface.co/docs/transformers/generation_strategies#contrastive-search)!

You can use contrastive search by providing `penalty_alpha` and `top_p` parameters to the `generate_kwargs` in the pipeline. You can read more about it [here](https://huggingface.co/blog/introducing-csearch). 🤗


In [9]:
for lang in list_of_languages:
    whisper_asr.model.config.forced_decoder_ids = (
        whisper_asr.tokenizer.get_decoder_prompt_ids(
            language=lang,
            task="transcribe"
            )
        )
    print(whisper_asr(
        next(iter(common_voice_en))["audio"]["array"],
        generate_kwargs =
         {
              "penalty_alpha": 0.6,
              "top_k": 5,
         }
        )["text"])

Reading metadata...: 16354it [00:00, 35945.16it/s]


 Joe Keaton verweigerte Filme und Buster hatte Reservations über die Medien. Und das heißt, wir haben eine Zeit, in der wir unser eigenes Interesse haben.


Reading metadata...: 16354it [00:00, 43245.81it/s]


 Joe Keaton disapprovò i film e Buster aveva anche riservazioni sui media. In una scelta di attrazione, i filmmi erano scegliuti da un gruppo d'autoressi che si chiamava Femme Femmes.


Reading metadata...: 16354it [00:00, 35353.34it/s]


 Joe Keaton aprovechó de los filmes y Buster también tenía reservaciones sobre la media.


Reading metadata...: 16354it [00:00, 73266.20it/s]


 Joe Keaton onvoldoende films en Buster had ook besluitingen over de medewerker. Ik ben geen kwaadaantje, maar ik heb een verhaal gehad over de medie.


Reading metadata...: 16354it [00:00, 34766.01it/s]


 Joe Keaton n'approuve pas les films et Buster avait également des préjugés sur le media.


Notice the subtle differences in the transcription, for your actual use-case, I'd recommend tuning these parameters a bit!

Good luck! 🤝